
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>



# De-Duping Data Lab

In this exercise, we're doing ETL on a file we've received from a customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL").
* The Social Security numbers aren't consistent either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

If all of the name fields match -- if you disregard character case -- then the birth dates and salaries are guaranteed to match as well,
and the Social Security Numbers *would* match if they were somehow put in the same format.

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lowercase, you haven't met this requirement.

<img src="https://files.training.databricks.com/images/icon_hint_32.png" alt="Hint"> The initial dataset contains 103,000 records.
The de-duplicated result has 100,000 records.

Next, write the results in **Delta** format as a **single data file** to the directory given by the variable **delta_dest_dir**.

<img src="https://files.training.databricks.com/images/icon_hint_32.png" alt="Hint"> Remember the relationship between the number of partitions in a DataFrame and the number of files written.

##### Methods
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/io.html" target="_blank">DataFrameReader</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html" target="_blank">DataFrame</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html" target="_blank">Built-In Functions</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/io.html" target="_blank">DataFrameWriter</a>

In [0]:
%run ../Includes/Classroom-Setup

Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03"

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)

Creating & using the schema "labuser6023680_cng7_da_asp" in the catalog "hive_metastore"...(0 seconds)

Predefined tables in "labuser6023680_cng7_da_asp":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks/_checkpoints

Setup completed (5 seconds)

P



It's helpful to look at the file first, so you can check the format with **`dbutils.fs.head()`**.

In [0]:
dbutils.fs.head(f"{DA.paths.datasets}/people/people-with-dups.txt")

[Truncated to first 65536 bytes]
Out[42]: 'firstName:middleName:lastName:gender:birthDate:salary:ssn\nEmanuel:Wallace:Panton:M:1988-03-04:101255:935-90-7627\nEloisa:Rubye:Cayouette:F:2000-06-20:204031:935-89-9009\nCathi:Svetlana:Prins:F:2012-12-22:35895:959-30-7957\nMitchel:Andres:Mozdzierz:M:1966-05-06:55108:989-27-8093\nAngla:Melba:Hartzheim:F:1938-07-26:13199:935-27-4276\nRachel:Marlin:Borremans:F:1923-02-23:67070:996-41-8616\nCatarina:Phylicia:Dominic:F:1969-09-29:201021:999-84-8888\nAntione:Randy:Hamacher:M:2004-03-05:271486:917-96-3554\nMadaline:Shawanda:Piszczek:F:1996-03-17:183944:963-87-9974\nLuciano:Norbert:Sarcone:M:1962-12-14:73069:909-96-1669\nNewton:Jose:Piacente:M:1969-11-05:264422:967-61-8575\nGretta:Jennefer:Dipinto:F:1921-10-28:118497:907-49-2510\nJenni:Janella:Mcquiston:F:1969-07-11:137418:932-55-7164\nMalena:Apryl:Kings:F:1980-01-08:98204:934-48-2334\nWilly:Russell:Merker:M:1967-10-10:41026:992-83-4829\nJami:Allen:Mulkerin:F:1958-08-08:236024:934-86-4126\nFernando:L

In [0]:
# TODO
from pyspark.sql.functions import lower, col

source_file = f"{DA.paths.datasets}/people/people-with-dups.txt"
delta_dest_dir = f"{DA.paths.working_dir}/people"

# In case it already exists
dbutils.fs.rm(delta_dest_dir, True)

# Complete your work here...
df = spark.read.option("header", "true").option("delimiter", ":").csv(source_file)

df = df.withColumn("firstName", lower(col("firstName")))
df = df.withColumn("middleName", lower(col("middleName")))
df = df.withColumn("lastName", lower(col("lastName")))

df_deduped = df.dropDuplicates(["firstName", "middleName", "lastName"])

In [0]:
display(df_deduped)

firstName,middleName,lastName,gender,birthDate,salary,ssn
aaron,alphonse,sterger,M,1969-03-12,103521,934-90-9344
aaron,alphonso,chocron,M,1945-06-30,116550,975-11-3909
aaron,andrea,mondloch,M,1937-12-02,75637,906-59-7221
aaron,barton,crasco,M,1986-11-21,298912,986-88-3115
aaron,brady,morgans,M,1935-10-25,283121,912-45-3172
aaron,cedrick,buzzi,M,1937-12-25,254596,963-70-8812
aaron,daniel,wildberger,M,1938-06-07,58049,968-88-1272
aaron,emile,boyster,M,1973-05-25,270994,925-70-5207
aaron,gaylord,santamarina,M,2009-12-01,298613,975-12-5330
aaron,jasper,sensor,M,1915-04-30,106207,928-37-9459


In [0]:
df_deduped.repartition(1).write.format("delta").mode("overwrite").save(delta_dest_dir)
# display(dbutils.fs.ls(delta_dest_dir))

In [0]:
# (deduped_df
#  .repartition(1)
#  .write
#  .mode("overwrite")
#  .format("delta")
#  .save(delta_dest_dir)
# )

display(dbutils.fs.ls(delta_dest_dir))

path,name,size,modificationTime
dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks/people/_delta_log/,_delta_log/,0,1709631269099
dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks/people/part-00000-821fe0d0-bd67-400d-9e0d-8b8e3a2be8be-c000.snappy.parquet,part-00000-821fe0d0-bd67-400d-9e0d-8b8e3a2be8be-c000.snappy.parquet,2905957,1709631264000





**CHECK YOUR WORK**

In [0]:
verify_files = dbutils.fs.ls(delta_dest_dir)
verify_delta_format = False
verify_num_data_files = 0
for f in verify_files:
    if f.name == "_delta_log/":
        verify_delta_format = True
    elif f.name.endswith(".parquet"):
        verify_num_data_files += 1

assert verify_delta_format, "Data not written in Delta format"
assert verify_num_data_files == 1, "Expected 1 data file written"

verify_record_count = spark.read.format("delta").load(delta_dest_dir).count()
assert verify_record_count == 100000, "Expected 100000 records in final result"

del verify_files, verify_delta_format, verify_num_data_files, verify_record_count
print("All test pass")

All test pass




## Clean up classroom
Run the cell below to clean up resources.

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the schema "labuser6023680_cng7_da_asp"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/labuser6023680@vocareum.com/apache-spark-programming-with-databricks"...(1 seconds)

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)


&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>